# This dataset is intentionally a subset of all of the US iTunes podcasts. 

- prepr
- postive 4,5
- negative 1,2
- -
- regex
- -
- word2vec
- topic modeling

本來就是針對genre去做 每個聽眾對於不同genre的看法不一（除非再針對某個genre去做)

## Import Package & Data

In [1]:
import spacy
import re
import pandas as pd
import en_core_web_sm
import en_core_web_md
import spacy
from scipy.spatial.distance import cosine
import numpy as np
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import f1_score

In [2]:
# import pandas as pd
# import sqlite3

# Read sqlite query results into a pandas DataFrame
# con = sqlite3.connect("database.sqlite")
# pod = pd.read_sql_query("SELECT * from podcasts", con)
# reviews = pd.read_sql_query("SELECT * from reviews", con)
# cat = pd.read_sql_query("SELECT * from categories", con)

In [3]:
df = pd.read_csv('podcast_sample.csv', lineterminator='\n', index_col = 0) 

In [4]:
df.head()

,podcast_id,title,content,rating,author_id,created_at,itunes_id,slug,itunes_url,podcast_title,category
0,b313ef8ef0d5b64290d3036ff1bbf2d2,감성 라디오 음악도시,미국 서부에 있는 유학생이에요. 성시경씨 제대 후 라디오 복귀만 기다려오다가 6 월...,5,664CCA7142E9AE8,2011-09-14T13:25:46-07:00,442838670,fm-%EC%9D%8C%EC%95%85%EB%8F%84%EC%8B%9C-%EC%A2...,https://podcasts.apple.com/us/podcast/fm-%EC%9...,FM 음악도시(종영),music
1,abfb842993be20d21bfae7103addc5e9,They’ve really cut back on the content this se...,Last season there was a new pod every 3-4 days...,1,AD790CE113DCBC1,2018-04-11T13:46:47-07:00,1015394113,the-good-phight-for-philadelphia-phillies-fans,https://podcasts.apple.com/us/podcast/the-good...,The Good Phight: for Philadelphia Phillies fans,sports
2,ebdf879a424547d01862a9bbba18a0f3,Good info. source...,Bob brings a lot of knowledge to any firearm d...,4,E223A4B2642C970,2010-01-19T08:11:43-07:00,333180229,handgun-world-podcast,https://podcasts.apple.com/us/podcast/handgun-...,Handgun World Podcast,news
3,ab2fdb7db023b223d870487165d11ff3,Mixed,They have lost much of thier credibility by de...,3,E1E7DBE750D119E,2021-01-28T12:21:49-07:00,971901464,wsj-opinion-potomac-watch,https://podcasts.apple.com/us/podcast/wsj-opin...,WSJ Opinion: Potomac Watch,news
4,ca601bd1524322d0527b16adf2738ff3,Try it now!,Even better than I expected. I was interested ...,5,D7CA4858AFA2CFC,2017-08-24T10:55:20-07:00,1257821731,conversations-with-people-who-hate-me,https://podcasts.apple.com/us/podcast/conversa...,Conversations with People Who Hate Me,society


In [77]:
good_reviews = df[df['rating']>= 4]
bad_reviews = df[df['rating']<=2]

In [78]:
# nltk
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
pt = PorterStemmer()

In [79]:
import nltk
nltk.download('punkt') # A popular NLTK sentence tokenizer
nltk.download('stopwords') # library of common English stopwords

[nltk_data] Downloading package punkt to /Users/Justin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Justin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Good Review

## Prepocessing

In [94]:
def review_clean(review):
    review = review.strip().lower()
    review = re.sub(r'\'ve|\'s|\'m|\'re|\d', '', review)
    review = re.sub(r'\b(pod|podcast|great|really|good|listen|episode|highly)+(s?|es?)\b','', review)
    review = re.sub(r'\b(listen|listens|listening|listened|forward|look forward|looking forward|loos forward)\b', '', review)
    review = re.sub(r'\b(love|like|best)+(s?|es?)\b','', review)

    
    word_tokens = word_tokenize(review)
    
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    word_tokens = filtered_sentence
#     word_tokens = [pt.stem(w) for w in word_tokens]
    word_tokens = [lemmatizer.lemmatize(w) for w in word_tokens]
    
    text_clean = ' '.join(word_tokens)
    return text_clean

In [95]:
good_reviews['content'] = good_reviews.apply(lambda x: review_clean(x['content']), axis=1)

<ipython-input-95-7cb41d7f3e84>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_reviews['content'] = good_reviews.apply(lambda x: review_clean(x['content']), axis=1)


In [96]:
vec = CountVectorizer(stop_words="english", binary=True).fit(good_reviews["content"])
bag_of_words = vec.transform(good_reviews["content"])
sum_words = bag_of_words.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
words_freq[0:50]

[('time', 2984),
 ('make', 2659),
 ('work', 2603),
 ('guy', 2499),
 ('story', 2449),
 ('way', 2306),
 ('thank', 2203),
 ('life', 2118),
 ('new', 2074),
 ('know', 1971),
 ('guest', 1970),
 ('people', 1933),
 ('thing', 1908),
 ('feel', 1707),
 ('hear', 1703),
 ('host', 1697),
 ('topic', 1561),
 ('fun', 1527),
 ('year', 1518),
 ('day', 1471),
 ('content', 1451),
 ('lot', 1432),
 ('enjoy', 1393),
 ('think', 1386),
 ('need', 1382),
 ('want', 1377),
 ('thanks', 1376),
 ('interesting', 1367),
 ('week', 1362),
 ('amazing', 1345),
 ('talk', 1317),
 ('favorite', 1284),
 ('funny', 1254),
 ('better', 1207),
 ('real', 1201),
 ('say', 1194),
 ('friend', 1171),
 ('world', 1126),
 ('information', 1094),
 ('fan', 1092),
 ('help', 1075),
 ('come', 1052),
 ('job', 1048),
 ('wait', 1035),
 ('recommend', 1015),
 ('going', 1014),
 ('voice', 1001),
 ('conversation', 979),
 ('interview', 976),
 ('entertaining', 933)]

In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

vectorizer = TfidfVectorizer(ngram_range=(3,3),
                             token_pattern=r'\b[a-zA-Z]{3,}\b',
                             max_df=0.4, max_features=200, stop_words=stopwords.words())

In [98]:
corpus = list(good_reviews["content"].values)

In [99]:
X = vectorizer.fit_transform(corpus)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)

/Users/Justin/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aldiz', 'baiknya', 'baizik', 'berkali', 'bukatzeko', 'edota', 'eze', 'ezpabere', 'ezpada', 'ezperen', 'gainera', 'gainerontzean', 'guztiz', 'hainbestez', 'horra', 'kali', 'kurangnya', 'mata', 'olah', 'ordea', 'osterantzean', 'printr', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(
/Users/Justin/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [100]:
pd.DataFrame(X.toarray(), columns=terms)

,advice information helpful,amazing amazing amazing,amazing story teller,appreciate hard work,appreciate time effort,asks question guest,awesome work guy,banter make laugh,black life matter,blah blah blah,...,waiting long time,watching youtube video,wide range topic,wide variety guest,wide variety topic,wish found sooner,wish give star,work day faster,wow world wow,wow wow wow
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28246,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
tf_idf = tf_idf.sum(axis=1)
score = pd.DataFrame(tf_idf, columns=["score"])
score.sort_values(by="score", ascending=False, inplace=True)

In [102]:
score.head(20)

,score
breath fresh air,59.836317
long time listener,36.313809
make laugh loud,28.050138
feel hanging friend,24.250959
real estate investing,16.662244
started month ago,15.329551
found month ago,14.696983
reason give star,14.659102
spot difference easy,14.158671
wide range topic,13.567425


### Topic Modeling

In [103]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer


review_vectorizer = TfidfVectorizer(ngram_range=(2,2),
                             min_df=0.01, max_df=0.4, stop_words="english")

X_review, review_terms = review_vectorizer.fit_transform(good_reviews.content), review_vectorizer.get_feature_names_out()
review_tf_idf = pd.DataFrame(X_review.toarray(), columns=review_terms)
print(f"Review TF-IDF: {review_tf_idf.shape}")
print(review_tf_idf.head(5))
review_tf_idf.head(5)

Review TF-IDF: (28249, 1)
   true crime
0         0.0
1         0.0
2         0.0
3         0.0
4         0.0


,true crime
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [104]:
nmf = NMF(n_components=3)
W_review = nmf.fit_transform(X_review)
H_review = nmf.components_
print(f"Original shape of X reviews is {X_review.shape}")
print(f"Decomposed W reviews matrix is {W_review.shape}")
print(f"Decomposed H reviews matrix is {H_review.shape}")

Original shape of X reviews is (28249, 1)
Decomposed W reviews matrix is (28249, 3)
Decomposed H reviews matrix is (3, 1)


In [105]:
from typing import List
import numpy as np

def get_top_tf_idf_tokens_for_topic(H: np.array, feature_names: List[str], num_top_tokens: int = 5):
  """
  Uses the H matrix (K components x M original features) to identify for each
  topic the most frequent tokens.
  """
  for topic, vector in enumerate(H):
    print(f"TOPIC {topic}\n")
    total = vector.sum()
    top_scores = vector.argsort()[::-1][:num_top_tokens]
    token_names = list(map(lambda idx: feature_names[idx], top_scores))
    strengths = list(map(lambda idx: vector[idx] / total, top_scores))
    
    for strength, token_name in zip(strengths, token_names):
      print(f"\b{token_name} ({round(strength * 100, 1)}%)\n")
    print(f"=" * 50)

get_top_tf_idf_tokens_for_topic(H_review, review_tf_idf.columns.tolist(), 5)
print(f"Reviews Topics:\n\n")
get_top_tf_idf_tokens_for_topic(H_review, review_tf_idf.columns.tolist(), 10)

TOPIC 0

true crime (100.0%)

TOPIC 1

true crime (100.0%)

TOPIC 2

true crime (100.0%)

Reviews Topics:


TOPIC 0

true crime (100.0%)

TOPIC 1

true crime (100.0%)

TOPIC 2

true crime (100.0%)



In [106]:
def get_top_documents_for_each_topic(W: np.array, documents: List[str], num_docs: int = 5):
    sorted_docs = W.argsort(axis=0)[::-1]
    top_docs = sorted_docs[:num_docs].T
    per_document_totals = W.sum(axis=1)
    for topic, top_documents_for_topic in enumerate(top_docs):
        print(f"Topic {topic}")
        for doc in top_documents_for_topic:
            score = W[doc][topic]
            percent_about_topic = round(score / per_document_totals[doc] * 100, 1)
            print(f"{percent_about_topic}%", documents[doc])
            print("=" * 50)

In [107]:
get_top_documents_for_each_topic(W_review, good_reviews.content.tolist())

Topic 0
100.0% hope true crime series . well produced . one annoying .
100.0% wow story , host , detective work journalism . need make true crime list missing .
100.0% true crime fan . also amazing comedy . guy . take two get joke , , ’ hooked . got huge backlog time ! lpotl ! ! !
100.0% started day ago already ! ’ stop ! content , insight , perspective true crime !
100.0% every true crime criticism narrator petty . get . challenge citric put voice know despise hearing voice . annoying review boiled n't someone tell story .
Topic 1
0.1% . go plenty detail case ; ’ heard several case , gained much clearer understanding tell . ’ objective fact-based . biggest issue lot true crime sensationalization speculation ; lay fact objective way . ’ recommend enough ! ! !
0.1% david ’ speaking story style . nice switch popular seem similar candor . come across caring empathetic think important . american , interesting hear canadian true crime .
0.1% true crime right . jasmyne - troy , ny
0.1% aweso

## Word2vec

In [108]:
import spacy
# load the language model, but we disable the ner (named entity recognition) and parser (dependency parser)
# since we don't need them for our use case to speed things up
nlp = spacy.load('en_core_web_md', disable = ['ner', 'parser'])

/Users/Justin/opt/anaconda3/lib/python3.8/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [109]:
import numpy as np
def process_text(text):
  """
  This function will use Spacy to perform stopword removal and lemmatization.
  """
  doc = nlp(text)
  processed_text = " ".join([token.lemma_ for token in doc if not token.is_stop])
  # this will get the word2vec embeddings for the processed text (the average of each token in the doc's word2vec embeddings)
  return np.array(nlp(processed_text).vector)

In [110]:
# use pandas' apply(...) method to apply this process_text function to each row's text field
good_reviews["vectors"] = good_reviews.content.apply(process_text)

<ipython-input-110-cc89d5e0bf97>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_reviews["vectors"] = good_reviews.content.apply(process_text)


In [111]:
# now we have for each document a numpy array of 300 numbers (its word2vec embeddings)
good_reviews["vectors"]

0        [0.005518933, -0.5161569, -0.559347, -0.536282...
2        [-0.51205826, -0.5742024, -1.2540163, 0.385765...
4        [1.2810528, -0.66930777, -0.19595638, -2.56741...
5        [0.0016238403, -0.28106058, -0.54703975, -1.69...
7        [0.8212379, -0.005584643, -1.3926145, -1.49905...
                               ...                        
49992    [0.70252043, -0.18376486, -0.09802313, -1.5482...
49993    [5.0891, -3.3753002, -4.2695, -4.8156, 3.8904,...
49994    [1.1265317, -0.6807913, -2.5832686, -0.7725373...
49996    [-0.56076795, -0.5616698, -0.32426855, -0.1006...
49999    [1.4425428, -1.4996988, -2.84881, -1.6160922, ...
Name: vectors, Length: 28249, dtype: object

# Bad Reviews

### Preprocessing

In [126]:
def review_clean(review):
    review = review.strip().lower()
    review = re.sub(r'\'ve|\'s|\'m|\'re|\d', '', review)
    review = re.sub(r'\b(pod|podcast|great|really|good|listen|episode|highly)+(s?|es?)\b','', review)
    review = re.sub(r'\b(listen|listens|listening|listened|forward|look forward|looking forward|loos forward)\b', '', review)
    review = re.sub(r'\b(love|like|best|hate|worst|bad)+(s?|es?)\b','', review)

    
    word_tokens = word_tokenize(review)
    
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    word_tokens = filtered_sentence
#     word_tokens = [pt.stem(w) for w in word_tokens]
    word_tokens = [lemmatizer.lemmatize(w) for w in word_tokens]
    
    text_clean = ' '.join(word_tokens)
    return text_clean

In [127]:
bad_reviews['content'] = bad_reviews.apply(lambda x: review_clean(x['content']), axis=1)

<ipython-input-127-58230dc2584c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bad_reviews['content'] = bad_reviews.apply(lambda x: review_clean(x['content']), axis=1)


In [128]:
vec = CountVectorizer(stop_words="english", binary=True).fit(bad_reviews["content"])
bag_of_words = vec.transform(bad_reviews["content"])
sum_words = bag_of_words.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
words_freq[0:50]

[('time', 2403),
 ('people', 2002),
 ('host', 1819),
 ('make', 1712),
 ('story', 1619),
 ('way', 1406),
 ('guy', 1389),
 ('know', 1366),
 ('think', 1313),
 ('want', 1311),
 ('talk', 1280),
 ('thing', 1218),
 ('say', 1187),
 ('better', 1158),
 ('used', 1106),
 ('need', 1098),
 ('sound', 1003),
 ('year', 986),
 ('feel', 960),
 ('hear', 940),
 ('content', 936),
 ('minute', 916),
 ('guest', 899),
 ('talking', 885),
 ('voice', 821),
 ('new', 815),
 ('hard', 812),
 ('come', 809),
 ('lot', 799),
 ('going', 789),
 ('stop', 783),
 ('interesting', 763),
 ('opinion', 747),
 ('annoying', 743),
 ('topic', 736),
 ('review', 734),
 ('fact', 723),
 ('point', 721),
 ('trying', 710),
 ('listener', 709),
 ('woman', 681),
 ('actually', 675),
 ('try', 658),
 ('life', 647),
 ('right', 643),
 ('fan', 638),
 ('thought', 626),
 ('work', 624),
 ('star', 612),
 ('long', 612)]

In [129]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

vectorizer = TfidfVectorizer(ngram_range=(3,3),
                             token_pattern=r'\b[a-zA-Z]{3,}\b',
                             max_df=0.4, max_features=200, stop_words=stopwords.words())

In [130]:
corpus = list(bad_reviews["content"].values)

In [131]:
X = vectorizer.fit_transform(corpus)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)

/Users/Justin/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aldiz', 'baiknya', 'baizik', 'berkali', 'bukatzeko', 'edota', 'eze', 'ezpabere', 'ezpada', 'ezperen', 'gainera', 'gainerontzean', 'guztiz', 'hainbestez', 'horra', 'kali', 'kurangnya', 'mata', 'olah', 'ordea', 'osterantzean', 'printr', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(
/Users/Justin/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [132]:
pd.DataFrame(X.toarray(), columns=terms)

,actual fact case,add insult injury,amount time word,angry loud spoiled,annoying vocal fry,anti law enforcement,asks question guest,audio quality horrible,audio quality poor,audio quality terrible,...,waste time woman,watch youtube video,wing talking point,wish give star,wish give zero,wolf sheep clothing,woman spend time,writing negative review,year ago started,year high school
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16447,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16449,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [133]:
tf_idf = tf_idf.sum(axis=1)
score = pd.DataFrame(tf_idf, columns=["score"])
score.sort_values(by="score", ascending=False, inplace=True)

In [134]:
score.head(20)

,score
long time listener,44.349002
blah blah blah,18.515546
give zero star,16.691483
spend time talking,13.985865
black life matter,13.915095
full body chill,13.679891
total waste time,11.000000
voice nail chalkboard,10.677957
social justice warrior,8.678575
stop interrupting guest,8.678575


### Topic Modeling

In [135]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer


review_vectorizer = TfidfVectorizer(ngram_range=(2,2),
                             min_df=0.01, max_df=0.4, stop_words="english")

X_review, review_terms = review_vectorizer.fit_transform(bad_reviews.content), review_vectorizer.get_feature_names_out()
review_tf_idf = pd.DataFrame(X_review.toarray(), columns=review_terms)
print(f"Review TF-IDF: {review_tf_idf.shape}")
print(review_tf_idf.head(5))
review_tf_idf.head(5)

Review TF-IDF: (16452, 3)
   true crime  want hear  waste time
0         0.0        0.0         0.0
1         0.0        0.0         0.0
2         0.0        0.0         0.0
3         0.0        0.0         0.0
4         0.0        0.0         0.0


,true crime,want hear,waste time
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


In [136]:
nmf = NMF(n_components=3)
W_review = nmf.fit_transform(X_review)
H_review = nmf.components_
print(f"Original shape of X reviews is {X_review.shape}")
print(f"Decomposed W reviews matrix is {W_review.shape}")
print(f"Decomposed H reviews matrix is {H_review.shape}")

Original shape of X reviews is (16452, 3)
Decomposed W reviews matrix is (16452, 3)
Decomposed H reviews matrix is (3, 3)


In [137]:
from typing import List
import numpy as np

def get_top_tf_idf_tokens_for_topic(H: np.array, feature_names: List[str], num_top_tokens: int = 5):
  """
  Uses the H matrix (K components x M original features) to identify for each
  topic the most frequent tokens.
  """
  for topic, vector in enumerate(H):
    print(f"TOPIC {topic}\n")
    total = vector.sum()
    top_scores = vector.argsort()[::-1][:num_top_tokens]
    token_names = list(map(lambda idx: feature_names[idx], top_scores))
    strengths = list(map(lambda idx: vector[idx] / total, top_scores))
    
    for strength, token_name in zip(strengths, token_names):
      print(f"\b{token_name} ({round(strength * 100, 1)}%)\n")
    print(f"=" * 50)

get_top_tf_idf_tokens_for_topic(H_review, review_tf_idf.columns.tolist(), 5)
print(f"Reviews Topics:\n\n")
get_top_tf_idf_tokens_for_topic(H_review, review_tf_idf.columns.tolist(), 10)

TOPIC 0

waste time (100.0%)

true crime (0.0%)

want hear (0.0%)

TOPIC 1

true crime (100.0%)

want hear (0.0%)

waste time (0.0%)

TOPIC 2

want hear (100.0%)

waste time (0.0%)

true crime (0.0%)

Reviews Topics:


TOPIC 0

waste time (100.0%)

true crime (0.0%)

want hear (0.0%)

TOPIC 1

true crime (100.0%)

want hear (0.0%)

waste time (0.0%)

TOPIC 2

want hear (100.0%)

waste time (0.0%)

true crime (0.0%)



In [138]:
def get_top_documents_for_each_topic(W: np.array, documents: List[str], num_docs: int = 5):
    sorted_docs = W.argsort(axis=0)[::-1]
    top_docs = sorted_docs[:num_docs].T
    per_document_totals = W.sum(axis=1)
    for topic, top_documents_for_topic in enumerate(top_docs):
        print(f"Topic {topic}")
        for doc in top_documents_for_topic:
            score = W[doc][topic]
            percent_about_topic = round(score / per_document_totals[doc] * 100, 1)
            print(f"{percent_about_topic}%", documents[doc])
            print("=" * 50)

In [139]:
get_top_documents_for_each_topic(W_review, bad_reviews.content.tolist())

Topic 0
100.0% ’ waste time prideful person
100.0% perhaps ’ ever . host juvenile annoying . shockingly , seem knowledge artist discussing popular music history . terrible waste time .
100.0% show started strong get past tell stretching could accomplished minute purpose ad space . random hearsay completely pointless recording junior detective waste time . get substance !
100.0% sadly mostly ad boring host story ... ghost story submitted caller . also waste time host read partial story , hoping people pay hear rest . disappointing !
100.0% sorry , ca n't take `` woe `` anymore . five year , nothing bunch navel gazing , narrow minded child-adults . growth , le le entertainment . sorry guy , waste time .
Topic 1
100.0% first , started pretty . mostly looking true crime fan . however , could longer stomach political propaganda took leave . wanted well done true crime show , another political commentary . disappointed .
100.0% first started simply true crime . get hear informative narrative

### Word2Vec

In [140]:
import spacy
# load the language model, but we disable the ner (named entity recognition) and parser (dependency parser)
# since we don't need them for our use case to speed things up
nlp = spacy.load('en_core_web_md', disable = ['ner', 'parser'])

/Users/Justin/opt/anaconda3/lib/python3.8/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['ner', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [141]:
import numpy as np
def process_text(text):
  """
  This function will use Spacy to perform stopword removal and lemmatization.
  """
  doc = nlp(text)
  processed_text = " ".join([token.lemma_ for token in doc if not token.is_stop])
  # this will get the word2vec embeddings for the processed text (the average of each token in the doc's word2vec embeddings)
  return np.array(nlp(processed_text).vector)

In [142]:
# use pandas' apply(...) method to apply this process_text function to each row's text field
bad_reviews["vectors"] = bad_reviews.content.apply(process_text)

<ipython-input-142-7dd07aa31ad1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bad_reviews["vectors"] = bad_reviews.content.apply(process_text)


In [143]:
# now we have for each document a numpy array of 300 numbers (its word2vec embeddings)
bad_reviews["vectors"]

1        [0.32942572, 0.998488, -0.54330236, 0.79712933...
6        [-0.86089176, -0.37215918, -1.6511924, -1.6320...
8        [1.7946743, 2.6819084, -0.10883585, -0.5083608...
10       [0.40497166, -0.05002085, -1.576666, -0.614707...
14       [-0.02933349, 0.56123495, -1.7128899, -1.30029...
                               ...                        
49987    [-0.1701629, -0.48754764, -1.2045877, -1.00413...
49988    [-1.3511939, -0.9265924, 1.070314, -0.48081094...
49995    [-1.7057341, -3.6644633, -7.7446094, 0.7407636...
49997    [-1.015794, -0.70189905, -0.41488737, -0.63473...
49998    [-1.1142291, -1.3235112, -3.483763, -1.5118328...
Name: vectors, Length: 16452, dtype: object